# Imports

In [1]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from langchain_community.vectorstores import Neo4jVector
from langchain_ollama import OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import TextLoader, PyMuPDFLoader
from langchain_core.runnables import RunnablePassthrough, RunnableConfig
from pprint import pprint
from langchain_neo4j.vectorstores.neo4j_vector import remove_lucene_chars

c:\Users\nevrohelios\Desktop\New folder\rag-institute\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

# Initiate Neo4j Database

In [5]:
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    database="neo4j",
)

# Load the `pdf` doc

In [11]:
loader = PyMuPDFLoader("data/AllSem23.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)

In [15]:
for doc in documents:
    doc.page_content = remove_lucene_chars(doc.page_content)
    doc.page_content = doc.page_content.replace("\n", " ")

In [17]:
pprint(documents[56].page_content)

('Maulana Abul Kalam Azad University of Technology  Formerly West Bengal '
 'University of Technology  Syllabus and Curricular Mapping for B. Tech. in '
 'Computer Science and Engineering Effective from Academic Session 2023 24 '
 'Page 43   81 of inherited attributes. 5 Type checking  4L  Type systems, '
 'Specification of a simple type checker, Equivalence of type expressions, '
 'Type conversions 4 6 Run time environments  5L  Source language issues  '
 'Activation trees, Control stack, scope of declaration, Binding of names , '
 'Storage organization  Subdivision of run time memory, Activation records , '
 'Storage allocation strategies, Parameter passing  call by value, call by '
 'reference, copy restore, call by name , Symbol tables, dynamic storage '
 'allocation techniques 5 7 Intermediate code generation  4L  Intermediate '
 'languages, Graphical representation, Three  address code, Implementation of '
 'three address statements  Quadruples, Triples, Indirect triples . 4 8 Co

In [57]:
print(len(documents))

105


## Define Entities

In [6]:
from typing import List

class Entities(BaseModel):
    """A generalized model for identifying key entities in various documents,
    such as research papers and academic syllabi."""

    person: List[str] = Field(
        description="Names of individuals, such as authors, professors, or cited researchers.",
        default_factory=list
    )
    organization: List[str] = Field(
        description="Organizations, including universities, departments, research institutes, or companies.",
        default_factory=list
    )
    topic: List[str] = Field(
        description="Key subjects, concepts, or topics discussed in the text.",
        default_factory=list
    )
    publication: List[str] = Field(
        description="Cited or mentioned publications, such as papers, books, or articles.",
        default_factory=list
    )
    location: List[str] = Field(
        description="Geographical or institutional locations mentioned.",
        default_factory=list
    )
    date: List[str] = Field(
        description="Specific dates or years relevant to the document's context.",
        default_factory=list
    )
    course: List[str] = Field(
        description="The names or codes of academic courses, if mentioned.",
        default_factory=list
    )
    assessment: List[str] = Field(
        description="Methods of evaluation, such as exams, quizzes, projects, or assignments.",
        default_factory=list
    )

In [7]:
allowed_nodes = list(Entities.__fields__.keys())
allowed_relationships = [
    "MENTIONS", "DESCRIBES", "CITES", "RELATED_TO", "PART_OF",
    "AUTHORED_BY", "USES_INSTRUMENT", "MEASURES", "LOCATED_IN",
    "STUDIES", "PUBLISHED_IN", "REFERENCES"
]

## `doc` to `graph doc`

In [ ]:
llm = ChatOllama(
    model="llama3.1:8b",
    temperature=0,
)
llm_transformer = LLMGraphTransformer(llm=llm,
                                      allowed_nodes=allowed_nodes,
                                      allowed_relationships=allowed_relationships,)
graph_docs = llm_transformer.convert_to_graph_documents(documents)

In [60]:
graph_docs[0]

GraphDocument(nodes=[Node(id='Maulana Abul Kalam Azad University Of Technology', type='Organization', properties={}), Node(id='West Bengal University Of Technology', type='Organization', properties={})], relationships=[], source=Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': '', 'creationdate': '2023-05-23T13:52:40+05:30', 'source': 'data/AllSem23.pdf', 'file_path': 'data/AllSem23.pdf', 'total_pages': 81, 'format': 'PDF 1.7', 'title': 'Syllabus & Curriculum Mapping B Tech (CSE) (1).docx', 'author': 'MAKAUT 10', 'subject': '', 'keywords': '', 'moddate': '2023-05-23T13:52:40+05:30', 'trapped': '', 'modDate': "D:20230523135240+05'30'", 'creationDate': "D:20230523135240+05'30'", 'page': 0, 'id': 'a664efaa6d7db0d85dcc362ab6ebb376'}, page_content='Maulana Abul Kalam Azad University of Technology  Formerly West Bengal University of Technology  Syllabus and Curricular Mapping for B. Tech. in Computer Science and Engineering Effective from Academic Session 2023 24 Page 1  

In [61]:
graph.query("MATCH (n) DETACH DELETE n")

[]

## add graph docs to neo4j

In [62]:
graph.add_graph_documents(
    graph_documents=graph_docs,
    baseEntityLabel=True,
    include_source=True
)

### Visialize the added `graph docs`

In [10]:
from yfiles_jupyter_graphs import GraphWidget

def show_graph():
    driver = GraphDatabase.driver(
        uri=os.getenv("NEO4J_URI"),
        auth=(os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))
    )
    with driver.session() as session:
        result = session.run("""
            MATCH (n)-[r]->(m)
            RETURN n, r, m
        """)
        widget = GraphWidget(graph=result.graph())
    return widget

show_graph()

show_graph()

GraphWidget(layout=Layout(height='800px', width='100%'))

## add vector embeddings

In [11]:
embedding_model = OllamaEmbeddings(
    model="nomic-embed-text:latest"
)

In [45]:
# graph.query("DROP INDEX vector IF EXISTS")
# graph.query(
#     "CREATE FULLTEXT INDEX documentFullTextIndex IF NOT EXISTS "
#     "FOR (n:Document) ON EACH [n.text, n.source, n.title]"
# )

[]

In [13]:
vector_index = Neo4jVector.from_existing_graph(
    embedding=embedding_model,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text", "source", "title"],
    embedding_node_property="embedding",
    keyword_index_name="documentFullTextIndex" 
)
vector_retriever = vector_index.as_retriever()

In [67]:
res = vector_retriever.invoke(
    "Automata syllabus?",
    top_k=5,
    similarity_threshold=0.7,
    include_embeddings=True
)
pprint(res[0].model_dump()["page_content"])

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: "CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN reduce(str='', k IN ['text', 'source', 'title'] | str + '\\n' + k + ': ' + coalesce(node[k], '')) AS text, n

('\n'
 'text: Maulana Abul Kalam Azad University of Technology  Formerly West Bengal '
 'University of Technology  Syllabus and Curricular Mapping for B. Tech. in '
 'Computer Science and Engineering Effective from Academic Session 2023 24 '
 'Page 33   81 COURSE OUTCOMES  COs  On completion of the course students will '
 'be able to Course Outcomes Details Action Verb Knowledge Level PCC CS403.CO1 '
 'Write a formal notation for strings, languages and machines. Write K6 PCC '
 'CS403.CO2 Design finite automata to accept a set of strings of a language. '
 'Design K6 PCC CS403.CO3 For a given language determine whether the given '
 'language is regular or not Determine K5 PCC CS403.CO4 Design context free '
 'grammars to generate strings of context free language. Design K6 PCC '
 'CS403.CO5 Determine equivalence of languages accepted by Push Down Automata '
 'and languages generated by context free grammars Determine K5 PCC CS403.CO6 '
 'Write the hierarchy of formal languages, grammars

# Prepare base prompt

In [14]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert in scientific papers. Your task is to extract entities from the text. "
        ),
        (
            "human",
            "Use the given format to extract information from the following"
            "input: {question}"
        )
    ]
)

entity_chain = prompt | llm.with_structured_output(Entities)

In [15]:
entities = entity_chain.invoke(
    {"question": "What is the syllabus of automata?"}
)

entities

Entities(person=[], organization=[], topic=['automata'], publication=[], location=[], date=[], course=['syllabus'], assessment=[])

# Full retriever

In [58]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

def generate_full_text_query(input: str) ->  str:
    words = [el for el in remove_lucene_chars(input).split() if el]
    if not words:
        return ""
    full_text_query = " AND ".join([f"{word}~2" for word in words])
    return full_text_query.strip()

def graph_retriever(question: str) -> str:
    """Retrieves relevant documents and related entities based on the question."""
    result = ""
    entities = entity_chain.invoke({"question": question}) 
    seen = set()
    for entity_type, value in entities.dict().items():
        if not value:
            continue
        value = value[0].strip()
        if not value or value in seen:
            continue  # Skip empty fields
        seen.add(value)

        fuzzy_query = generate_full_text_query(value)

        response = graph.query(
            """
            CALL db.index.fulltext.queryNodes('documentFullTextIndex', $query, {limit: 5})
            YIELD node, score

            CALL {
            WITH node
            OPTIONAL MATCH (node)-[:MENTIONS|RELATED_TO*1..2]-(doc:Document)
            RETURN DISTINCT doc.title AS title, doc.text AS text, doc.page AS page
            }

            RETURN title, text, page, score
            ORDER BY score DESC
            """,
            {"query": fuzzy_query}
        )

        for record in response:
            result += f"Matched Field: {entity_type}\n"
            result += f"Entity: {value}\n"
            result += f"Title: {record['title']}\n"
            result += f"Page: {record['page']}\n"
            result += f"Text: {record['text']}\n"
            result += f"Score: {record['score']:.2f}\n\n"

    return result.strip()



# generate_full_text_query("What the contents of this paper say?")
graph_retriever("What are the core objectives of the Engineering Physics course in Semester I?")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node) { ... }} {position: line: 5, column: 13, offset: 135} for query: "\n            CALL db.index.fulltext.queryNodes('documentFullTextIndex', $query, {limit: 5})\n            YIELD node, score\n\n            CALL {\n            WITH node\n            OPTIONAL MATCH (node)-[:MENTIONS|RELATED_TO*1..2]-(doc:Document)\n            RETURN DISTINCT doc.title AS title, doc.text AS text, doc.page AS page\n            }\n\n            RETURN title, text, page, score\n            ORDER BY score DESC\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature

"Matched Field: course\nEntity: Engineering Physics\nTitle: Syllabus & Curriculum Mapping B Tech (CSE) (1).docx\nPage: 1\nText: Maulana Abul Kalam Azad University of Technology  Formerly West Bengal University of Technology  Syllabus and Curricular Mapping for B. Tech. in Computer Science and Engineering Effective from Academic Session 2023 24 Page 2   81 CO Statements and CO PO Mapping for Courses of New Syllabus  SEMESTER – I THEORY COURSE OUTCOMES  COs  On completion of the course students will be able to Course Outcomes Details Action Verb Knowledge Level BS M101.CO1 Learn the basic mathematical tools to deal with problems of engineering sciences. Learn Level I BS M101.CO2 Understand properties and application of Calculus and Linear Algebra . Understand Level II BS M101.CO3 Analyze of physical or engineering problems. Analyze Level IV BS M101.CO4 Acquire problem solving skills related to engineering science. Acquire Level II BS M101.CO5 Apply Calculus and Linear Algebra in real lif

In [59]:
def full_retriever(query: str):
    """Retrieves relevant documents and entities based on the query."""
    graph_data = graph_retriever(question=query)
    vector_data = [el.model_dump()["page_content"] for el in vector_retriever.invoke(query)]

    final_data = f"""Graph Data:\n{graph_data}\n\nVector Data:\n
                  {"\nDocument".join(vector_data)}"""
    
    return final_data.strip()

In [60]:
op = full_retriever("What is the syllabus of automata?")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node) { ... }} {position: line: 5, column: 13, offset: 135} for query: "\n            CALL db.index.fulltext.queryNodes('documentFullTextIndex', $query, {limit: 5})\n            YIELD node, score\n\n            CALL {\n            WITH node\n            OPTIONAL MATCH (node)-[:MENTIONS|RELATED_TO*1..2]-(doc:Document)\n            RETURN DISTINCT doc.title AS title, doc.text AS text, doc.page AS page\n            }\n\n            RETURN title, text, page, score\n            ORDER BY score DESC\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature

# Examples

In [61]:
template = """Answer the question based on the provided data.
Context: 
{context}

Question: {question}

Use the context to answer the question as accurately as possible. Be concise and to the point.

Answer:
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": full_retriever,
        "question": RunnablePassthrough(),
    } 
    | prompt
    | llm
    | StrOutputParser()
)

In [62]:
output = chain.invoke(input="What is the syllabus of automata?")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node) { ... }} {position: line: 5, column: 13, offset: 135} for query: "\n            CALL db.index.fulltext.queryNodes('documentFullTextIndex', $query, {limit: 5})\n            YIELD node, score\n\n            CALL {\n            WITH node\n            OPTIONAL MATCH (node)-[:MENTIONS|RELATED_TO*1..2]-(doc:Document)\n            RETURN DISTINCT doc.title AS title, doc.text AS text, doc.page AS page\n            }\n\n            RETURN title, text, page, score\n            ORDER BY score DESC\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature

In [63]:
from IPython.display import Markdown as md
md(output)

The syllabus for Automata Theory includes:

* Introduction to alphabet, languages, and grammars
* Regular languages and finite automata (DFA and NFA)
* Context-free languages and pushdown automata (PDA)
* Context-sensitive languages and linear bounded automata
* Turing machines (TM) and their variants

The course also covers topics such as:

* Formal language hierarchy
* Computability and decidability
* Equivalence of languages accepted by PDA and context-free grammars
* Designing finite automata to accept a set of strings
* Writing formal notations for strings, languages, and machines.

In [64]:
md(chain.invoke(input="what is this doc about?"))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: "CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN reduce(str='', k IN ['text', 'source', 'title'] | str + '\\n' + k + ': ' + coalesce(node[k], '')) AS text, n

This document appears to be a syllabus for the Bachelor of Technology in Computer Science and Engineering program at Maulana Abul Kalam Azad University of Technology, outlining the course structure and curriculum for the 2023-24 academic session.

In [65]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import (
    context_recall,
)
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness
from ragas import EvaluationDataset

In [66]:
sample_queries = [
    "What are the core objectives of the Engineering Physics course in Semester I?",
    "What mathematical concepts are taught in Mathematics–IA (BS-M101)?",
    "How is the course outcome of BS-M201 aligned with program outcomes?",
    "What topics are included in the Programming for Problem Solving (ES-CS201) course?",
    "What are the key course outcomes of Basic Electrical Engineering (ES-EE101)?",
    "What are the program-specific outcomes (PSOs) for CSE students as per MAKAUT syllabus?",
    "What skills are expected from students after completing Physics-I Laboratory (BS-PH191)?",
    "What is taught under the Chemistry-I (BS-CH201) course in Semester II?",
    "What is the significance of the CO-PO mapping in the syllabus document?",
    "Which programming paradigms are covered in Data Structures and Algorithms (PCC-CS301)?"
]

expected_responses = [
    "The core objectives include applying basic concepts of mechanics and optics, differentiating between classical and quantum physics, and analyzing electromagnetic laws and quantum mechanics principles.",
    "Mathematics–IA covers integration and differentiation techniques, linear algebra, vector spaces, eigenvalues and eigenvectors, and inner product spaces, along with their applications in engineering.",
    "BS-M201 (Mathematics-IIA) outcomes are highly aligned with PO1 to PO4, emphasizing analytical, problem-solving, and application skills in Linear Algebra, ODE, and numerical analysis.",
    "Topics include algorithms, flowcharts, C programming basics, conditional statements, loops, arrays, functions, recursion, structures, pointers, and file handling.",
    "Key outcomes include understanding and analyzing electric and magnetic circuits, electrical machines, transformers, power converters, and electrical installations.",
    "CSE graduates are expected to design computational solutions using mathematical and programming foundations, and contribute effectively in interdisciplinary teams while pursuing lifelong learning and innovative research.",
    "Students will be able to observe and measure physical properties using instruments, apply quantum principles, and calculate constants like Planck's and Stefan's from experiments.",
    "The course includes thermodynamics, electrochemistry, kinetics, stereochemistry, spectroscopy, periodic properties, and synthesis of drug molecules.",
    "CO-PO mapping ensures each course outcome aligns with the broader program outcomes, allowing structured curriculum development and measurable academic impact.",
    "The course emphasizes procedural and abstract data type paradigms, including stacks, queues, linked lists, trees, sorting, searching, and hashing algorithms."
]



dataset = []

for query,reference in zip(sample_queries,expected_responses):

    relevant_docs_combined = full_retriever(query)
    graph_part = relevant_docs_combined.split("Vector Data:")[0].replace("Graph Data:", "").strip()
    vector_part = relevant_docs_combined.split("Vector Data:")[1].strip()
    retrieved_contexts = [graph_part, vector_part] if graph_part and vector_part else [relevant_docs_combined]
    response = chain.invoke(input=query)
    dataset.append(
        {
            "user_input":query,
            "retrieved_contexts":retrieved_contexts,
            "response":response,
            "reference":reference
        }
    )

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node) { ... }} {position: line: 5, column: 13, offset: 135} for query: "\n            CALL db.index.fulltext.queryNodes('documentFullTextIndex', $query, {limit: 5})\n            YIELD node, score\n\n            CALL {\n            WITH node\n            OPTIONAL MATCH (node)-[:MENTIONS|RELATED_TO*1..2]-(doc:Document)\n            RETURN DISTINCT doc.title AS title, doc.text AS text, doc.page AS page\n            }\n\n            RETURN title, text, page, score\n            ORDER BY score DESC\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature

In [67]:
evaluation_dataset = EvaluationDataset.from_list(dataset)

In [68]:
llm

ChatOllama(model='llama3.1:8b', temperature=1e-08)

In [69]:
evaluator_llm = LangchainLLMWrapper(llm)

result = evaluate(dataset=evaluation_dataset,metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],llm=evaluator_llm)
result

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

Exception raised in Job[0]: TimeoutError()
Exception raised in Job[1]: TimeoutError()
Exception raised in Job[2]: TimeoutError()
Exception raised in Job[4]: TimeoutError()
Exception raised in Job[5]: TimeoutError()
Exception raised in Job[6]: TimeoutError()
Exception raised in Job[7]: TimeoutError()
Exception raised in Job[8]: TimeoutError()
Exception raised in Job[9]: TimeoutError()
Exception raised in Job[10]: TimeoutError()
Exception raised in Job[11]: TimeoutError()
Exception raised in Job[12]: TimeoutError()
Exception raised in Job[13]: TimeoutError()
Exception raised in Job[14]: TimeoutError()
Exception raised in Job[15]: TimeoutError()
Exception raised in Job[16]: TimeoutError()
Exception raised in Job[17]: TimeoutError()
Exception raised in Job[18]: TimeoutError()
Exception raised in Job[19]: TimeoutError()
Exception raised in Job[20]: TimeoutError()
Exception raised in Job[21]: TimeoutError()
Exception raised in Job[22]: TimeoutError()
Exception raised in Job[23]: TimeoutError

{'context_recall': 0.3333, 'faithfulness': nan, 'factual_correctness(mode=f1)': nan}